# Initialization

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from result_saver import SaverProvider

provider = SaverProvider()

# Initialize simulator

In [3]:
from soft_info import RepCodeIQSimulator

DEVICE = 'ibm_sherbrooke'
DISTANCE = 3
ROUNDS = 3
OTHER_DATE = '2023-10-27'
_RESETS = True
LOGICAL = 0 # NOT NEEDED FOR EXTREME IQ BCS HARDCODED 0

_is_hex = True
if DEVICE == 'ibmq_mumbai':
    _is_hex = False

# Initialize simulator
simulator = RepCodeIQSimulator(provider, DISTANCE, ROUNDS, DEVICE, _is_hex=_is_hex, _resets = _RESETS, other_date=OTHER_DATE)

Found jobs for backend ibm_sherbrooke with closest execution date 2023-10-27 08:32:22.841567+00:00.
Found jobs for backend ibm_sherbrooke with closest execution date 2023-10-27 08:32:22.841567+00:00.
Searching for ibm_sherbrooke and 23.10.27_07h46_300pts_2std


# Get IQ data

In [4]:
SHOTS = int(1e4)
NOISE_LIST = [3e-2, 0.8e-2, 1e-2, 3e-2] # [two-qubit-fidelity, reset error, measurement error, idle error]
# NOISE_LIST = None
P_AMBIG = 0.1

IQ_data = simulator.generate_IQ(SHOTS, noise_list=NOISE_LIST)
IQ_data_extreme = simulator.generate_extreme_IQ(SHOTS, P_AMBIG, noise_list=NOISE_LIST)

100%|██████████| 255/255 [00:00<00:00, 444.73it/s]


# Decode using predecoder

In [5]:
import pymatching
import stim
import cpp_soft_info
import numpy as np

model = simulator.stim_circ.detector_error_model(decompose_errors=False)
matching = pymatching.Matching.from_detector_error_model(model)

In [38]:
_DETAILED = False  

result_predecoder = cpp_soft_info.decode_time_nn_predecode_grid(model, IQ_data, ROUNDS, int(LOGICAL), _RESETS, simulator.qubit_mapping, simulator.grid_dict,
                                           simulator.processed_scaler_dict, _detailed=_DETAILED, threshold=0.30)


print(f"num_errors: {result_predecoder.decode_result.num_errors}")
print(f"mean num removed edges: {np.mean(result_predecoder.nb_rm_edges)}")
print(f"num removed edges: {np.sum(result_predecoder.nb_rm_edges)}")

num_errors: 386
mean num removed edges: 0.0135
num removed edges: 135


In [7]:
# Soft Grid info

result = cpp_soft_info.decode_IQ_fast(model, IQ_data,
                                           ROUNDS, int(LOGICAL), _RESETS, simulator.qubit_mapping, simulator.grid_dict,
                                           simulator.processed_scaler_dict, _detailed=_DETAILED, nb_intervals=-1)

print(f"num_errors: {result.num_errors}")

num_errors: 348


In [8]:
# Hard decoder
p_meas = -1

result_informed = cpp_soft_info.decode_IQ_shots_flat_informed(model, IQ_data,
                                           ROUNDS, int(LOGICAL), _RESETS, simulator.qubit_mapping, simulator.grid_dict, simulator.processed_scaler_dict,
                                           p_data = -1, p_mixed = -1, p_meas = p_meas, common_measure=-1, _detailed=_DETAILED, _ntnn_edges = False)

print(f"num_errors: {result_informed.num_errors}")

num_errors: 391
